<a href="https://colab.research.google.com/github/LEGEND2310/Market-Analysis/blob/main/Algorithmic_Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Method 1

Tutorial: https://medium.datadriveninvestor.com/predicting-the-stock-market-with-python-bba3cf4c56ef

In [ ]:
!pip install yfinance
!pip install ta
!pip install fastai
!pip install plotly

In [ ]:
import sys
import yfinance as yf

# Import the library that does dataframe management
import pandas as pd # Library that manages dataframes
import numpy as np

from ta import add_all_ta_features

import fastai.tabular

import plotly.graph_objs as go

from sklearn.metrics import mean_squared_error # Install error metrics 
from sklearn.linear_model import LinearRegression # Install linear regression model
from sklearn.neural_network import MLPRegressor # Install ANN model 
from sklearn.preprocessing import StandardScaler # to scale for ann

In [ ]:
# Ensure column types are correct

def CorrectColumnTypes(df):
  # Input: dataframe 
  # ouptut: dataframe (with column types changed)

  # Numbers
  for col in df.columns[1:80]:
      df[col] = df[col].astype('float')

  for col in df.columns[-10:]:
      df[col] = df[col].astype('float')

  # Categories 
  for col in df.columns[80:-10]:
      df[col] = df[col].astype('category')

  return df

In [ ]:
# Create the lags 
def CreateLags(df,lag_size):
  # inputs: dataframe , size of the lag (int)
  # ouptut: dataframe ( with extra lag column), shift size (int)

  # add lag
  shiftdays = lag_size
  shift = -shiftdays
  df['Close_lag'] = df['Close'].shift(shift)
  return df, shift

In [ ]:
# Split the testing and training data 
def SplitData(df, train_pct, shift):
  # inputs: dataframe , training_pct (float between 0 and 1), size of the lag (int)
  # ouptut: x train dataframe, y train data frame, x test dataframe, y test dataframe, train data frame, test dataframe

  train_pt = int(len(df)*train_pct)
  
  train = df.iloc[:train_pt,:]
  test = df.iloc[train_pt:,:]
  
  x_train = train.iloc[:shift,1:-1]
  y_train = train['Close_lag'][:shift]
  x_test = test.iloc[:shift,1:-1]
  y_test = test['Close'][:shift]

  return x_train, y_train, x_test, y_test, train, test

In [ ]:
# Function to make the plots
def PlotModelResults_Plotly(train, test, pred, ticker, w, h, shift_days,name):
  # inputs: train dataframe, test dataframe, predicted value (list), ticker ('string'), width (int), height (int), shift size (int), name (string)
  # output: None

  # Create lines of the training actual, testing actual, prediction 
  D1 = go.Scatter(x=train.index,y=train['Close'],name = 'Train Actual') # Training actuals
  D2 = go.Scatter(x=test.index[:shift],y=test['Close'],name = 'Test Actual') # Testing actuals
  D3 = go.Scatter(x=test.index[:shift],y=pred,name = 'Our Prediction') # Testing predction

  # Combine in an object  
  line = {'data': [D1,D2,D3],
          'layout': {
              'xaxis' :{'title': 'Date'},
              'yaxis' :{'title': '$'},
              'title' : name + ' - ' + tickerSymbol + ' - ' + str(shift_days)
          }}
  # Send object to a figure 
  fig = go.Figure(line)

  # Show figure
  fig.show()

In [ ]:
def LinearRegression_fnc(x_train,y_train, x_test, y_test):
  #inputs: x train data, y train data, x test data, y test data (all dataframe's)
  # output: the predicted values for the test data (list)
  
  lr = LinearRegression()
  lr.fit(x_train,y_train)
  lr_pred = lr.predict(x_test)
  lr_MSE = mean_squared_error(y_test, lr_pred)
  lr_R2 = lr.score(x_test, y_test)
  print('Linear Regression R2: {}'.format(lr_R2))
  print('Linear Regression MSE: {}'.format(lr_MSE))

  return lr_pred

In [ ]:
# ANN Function 

def ANN_func(x_train,y_train, x_test, y_test):

  # Scaling data
  scaler = StandardScaler()
  scaler.fit(x_train)
  x_train_scaled = scaler.transform(x_train)
  x_test_scaled = scaler.transform(x_test)


  MLP = MLPRegressor(random_state=1, max_iter=1000, hidden_layer_sizes = (100,), activation = 'identity',learning_rate = 'adaptive').fit(x_train_scaled, y_train)
  MLP_pred = MLP.predict(x_test_scaled)
  MLP_MSE = mean_squared_error(y_test, MLP_pred)
  MLP_R2 = MLP.score(x_test_scaled, y_test)

  print('Muli-layer Perceptron R2 Test: {}'.format(MLP_R2))
  print('Multi-layer Perceptron MSE: {}'.format(MLP_MSE))

  return MLP_pred

In [ ]:
def CalcProfit(test_df,pred,j):
  pd.set_option('mode.chained_assignment', None)
  test_df['pred'] = np.nan
  test_df['pred'].iloc[:-j] = pred
  test_df['change'] = test_df['Close_lag'] - test_df['Close'] 
  test_df['change_pred'] = test_df['pred'] - test_df['Close'] 
  test_df['MadeMoney'] = np.where(test_df['change_pred']/test_df['change'] > 0, 1, -1) 
  test_df['profit'] = np.abs(test['change']) * test_df['MadeMoney']
  profit_dollars = test['profit'].sum()
  print('Would have made: $ ' + str(round(profit_dollars,1)))
  profit_days = len(test_df[test_df['MadeMoney'] == 1])
  print('Percentage of good trading days: ' + str( round(profit_days/(len(test_df)-j),2))     )

  return test_df, profit_dollars

In [ ]:
# Choose your ticker
tickerSymbol = "AMZN"

# Choose date range - format should be 'YYYY-MM-DD' 
startDate = '2015-04-01' # as strings
endDate = '2021-10-01' # as strings

In [ ]:
# Create ticker yfinance object
tickerData = yf.Ticker(tickerSymbol)

# Create historic data dataframe and fetch the data for the dates given. 
df = tickerData.history(start = startDate, end = endDate)

# Print statement showing the download is done

# Show what the first 5 rows of the data frame
# Note the dataframe has:
#   - Date (YYYY-MM-DD) as an index
#   - Open (price the stock started as)
#   - High (highest price stock reached that day)
#   - Low (lowest price stock reached that day)
#   - Close (price the stock ended the day as)
#   - Volume (how many shares were traded that day)
#   - Dividends (any earnings shared to shareholders)
#   - Stock Splits (any stock price changes)

print('-----------------------')
print('Done!')
df.head()

-----------------------
Done!


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2015-04-01,372.100006,373.160004,368.339996,370.260010,2458100,0,0
2015-04-02,370.500000,373.279999,369.000000,372.250000,1875300,0,0
2015-04-06,370.100006,380.200012,369.359985,377.040009,3050700,0,0
2015-04-07,376.149994,379.309998,374.029999,374.410004,1954900,0,0
2015-04-08,374.660004,381.579987,374.649994,381.200012,2636400,0,0


In [ ]:
# Change the date column to a pandas date time column 

# Define string format
date_change = '%Y-%m-%d'

# Create a new date column from the index
df['Date'] = df.index

# Perform the date type change
df['Date'] = pd.to_datetime(df['Date'], format = date_change)

# Create a variable that is the date column
Dates = df['Date']

In [ ]:
# Add all technical analysis to the dataframe we've already loaded
df = add_all_ta_features(df, "Open", "High", "Low", "Close", "Volume", fillna=True) 

print(df.columns)

/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning:

invalid value encountered in double_scalars



Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Date', 'volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi',
       'volume_mfi', 'volume_em', 'volume_sma_em', 'volume_vpt', 'volume_nvi',
       'volume_vwap', 'volatility_atr', 'volatility_bbm', 'volatility_bbh',
       'volatility_bbl', 'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
       'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl',
       'volatility_kcw', 'volatility_kcp', 'volatility_kchi',
       'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'volatility_dcm',
       'volatility_dcw', 'volatility_dcp', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow', 'trend_adx',
       'trend_adx_pos', 'trend_adx_neg', 'trend_vortex_ind_pos',
       'trend_vortex_ind_neg', 'trend_vortex_ind_diff', 'trend_trix',
       'trend_mass_index', 'trend_cci', 'trend_

In [ ]:
# Define the date parts 
fastai.tabular.add_datepart(df,'Date', drop = 'True')

# Ensure the correct format
df['Date'] = pd.to_datetime(df.index.values, format = date_change)

# Add the date parts
fastai.tabular.add_cyclic_datepart(df, 'Date', drop = 'True')

/usr/local/lib/python3.7/dist-packages/fastai/tabular/transform.py:63: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



,Open,High,Low,Close,Volume,Dividends,Stock Splits,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,...,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,weekday_cos,weekday_sin,day_month_cos,day_month_sin,month_year_cos,month_year_sin,day_year_cos,day_year_sin
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-04-01,372.100006,373.160004,368.339996,370.260010,2458100,0,0,-4.997689e+05,2458100,-0.203315,0.000000e+00,50.000000,0.000000,0.000000,-1.897081e+06,1000.000000,370.586670,0.000000,370.260010,370.260010,370.260010,0.000000,0.000000,0.0,0.0,370.586670,375.406677,365.766663,2.601285,0.466114,0.0,0.0,368.339996,373.160004,370.750000,1.301790,0.398342,0.000000,0.000000,0.000000,...,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39.834243,39.834243,-60.165757,0.000000,370.260010,0.000000,0.000000,0.000000,0.000000,-77.396256,0.000000,0.000000,2015,4,14,1,2,91,False,True,False,True,False,False,1427846400,-0.222521,0.974928,1.000000,0.000000,6.123234e-17,1.000000,0.021516,0.999769
2015-04-02,370.500000,373.279999,369.000000,372.250000,1875300,0,0,4.729342e+05,4333400,0.109137,3.731829e+06,100.000000,89.009594,89.009594,-1.892398e+06,1005.374575,370.986245,0.000000,371.255005,373.244995,369.265015,1.072034,0.750000,0.0,0.0,371.048335,375.598338,366.498332,2.452512,0.632051,0.0,0.0,368.339996,373.279999,370.809998,1.330622,0.791498,0.000000,0.158746,0.031749,...,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,35.714262,79.149833,59.492038,-20.850167,0.000000,371.181369,0.000000,-1.925156,-0.385031,-1.540125,0.537458,0.536018,0.537458,2015,4,14,2,3,92,False,False,False,False,False,False,1427932800,-0.900969,0.433884,0.978148,0.207912,6.123234e-17,1.000000,0.004304,0.999991
2015-04-06,370.100006,380.200012,369.359985,377.040009,3050700,0,0,1.745000e+06,7384100,0.236319,5.286264e+06,100.000000,1293.397946,691.203770,4.933449e+04,1005.374575,372.864850,0.000000,373.183339,378.874370,367.492308,3.049992,0.838837,0.0,0.0,372.543335,379.190013,365.896657,3.568271,0.838265,0.0,0.0,368.339996,380.200012,374.270004,3.178067,0.733558,0.000000,0.663419,0.158083,...,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,54.814469,73.355823,64.113299,-26.644177,0.000000,373.858211,0.000000,0.462142,-0.215597,0.677739,1.286772,1.278563,1.831145,2015,4,15,6,0,96,False,False,False,False,False,False,1428278400,1.000000,0.000000,0.500000,0.866025,6.123234e-17,1.000000,-0.064508,0.997917
2015-04-07,376.149994,379.309998,374.029999,374.410004,1954900,0,0,7.149045e+04,5429200,0.007655,3.796599e+06,100.000000,510.470841,630.959460,2.561934e+04,998.361685,373.503676,0.000000,373.490005,378.531771,368.448240,2.699813,0.591238,0.0,0.0,373.386668,379.691676,367.081660,3.377200,0.581153,0.0,0.0,368.339996,380.200012,374.270004,3.175457,0.511804,0.000000,0.841457,0.294758,...,0.000000,70.092268,0.000000,0.000000,0.000000,97.182849,44.845416,51.180432,67.895362,-48.819568,0.000000,374.115947,0.000000,-1.271489,-0.426775,-0.844714,-0.697540,-0.699984,1.120832,2015,4,15,7,1,97,False,False,False,False,False,False,1428364800,0.623490,0.781831,0.309017,0.951057,6.123234e-17,1.000000,-0.081676,0.996659
2015-04-08,374.660004,381.579987,3

In [ ]:
# Define key model parameters

# Set days out to predict 
shifts = [50,100,200]

# Set a training percentage
train_pct = .75

# Plotting dimensions
w = 16 # width
h = 4 # height

In [ ]:
# Go through each shift....

for j in shifts: 
  print(str(j) + ' days out:')
  print('------------')
  df_lag, shift = CreateLags(df,j)
  df_lag = CorrectColumnTypes(df_lag)
  x_train, y_train, x_test, y_test, train, test = SplitData(df, train_pct, shift)

  # Linear Regression
  print("Linear Regression")
  lr_pred = LinearRegression_fnc(x_train,y_train, x_test, y_test)
  test2, profit_dollars = CalcProfit(test,lr_pred,j)
  PlotModelResults_Plotly(train, test, lr_pred, tickerSymbol, w, h, j, 'Linear Regression')

  # Artificial Neuarl Network 
  print("ANN")
  MLP_pred = ANN_func(x_train,y_train, x_test, y_test)
  test2, profit_dollars = CalcProfit(test,MLP_pred,j)
  PlotModelResults_Plotly(train, test, MLP_pred, tickerSymbol, w, h, j, 'ANN')
  print('------------')

50 days out:
------------
Linear Regression
Linear Regression R2: -0.15978829261968053
Linear Regression MSE: 262650.6941946635
Would have made: $ 45162.8
Percentage of good trading days: 0.55


ANN
Muli-layer Perceptron R2 Test: 0.07433861622367466
Multi-layer Perceptron MSE: 209629.29750642926
Would have made: $ 50586.6
Percentage of good trading days: 0.59


------------
100 days out:
------------
Linear Regression
Linear Regression R2: -3.9834886767544955
Linear Regression MSE: 1117856.1503306984
Would have made: $ -35379.8
Percentage of good trading days: 0.27


ANN


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.



...

------------
200 days out:
------------
Linear Regression
Linear Regression R2: -0.7015224784541347
Linear Regression MSE: 432544.95325035014
Would have made: $ -4337.3
Percentage of good trading days: 0.3


ANN


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.



Muli-layer Perceptron R2 Test: -1.0828326446948084
Multi-layer Perceptron MSE: 529478.017678803
Would have made: $ 1054.3
Percentage of good trading days: 0.3


------------


#Method 2

##Index Funds

In [ ]:
!pip install xlsxwriter

     |████████████████████████████████| 148 kB 28.9 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [ ]:
stocks = pd.read_csv("sp_500_stocks.csv")

In [ ]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [ ]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()

In [ ]:
data

{'avgTotalVolume': 85292346,
 'calculationPrice': 'close',
 'change': 2.97,
 'changePercent': 0.02069,
 'close': 145.73,
 'closeSource': 'liffoaic',
 'closeTime': 1668302375403,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 149.14,
 'delayedPriceTime': 1639217275459,
 'extendedChange': 0.19,
 'extendedChangePercent': 0.00126,
 'extendedPrice': 144.78,
 'extendedPriceTime': 1703972469621,
 'high': 148.68,
 'highSource': 'e dt a5e1nelecmrii pydu',
 'highTime': 1670281466176,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 149.9,
 'iexCloseTime': 1639573288447,
 'iexLastUpdated': 1645658854740,
 'iexMarketPercent': 0.013379044926881599,
 'iexOpen': 147.9,
 'iexOpenTime': 1693197681748,
 'iexRealtimePrice': 146.5,
 'iexRealtimeSize': 10,
 'iexVolume': 967795,
 'isUSMarketOpen': False,
 'lastTradeTime': 1650418571185,
 'latestPrice': 148.93,
 'latestSource': 'Close',
 'latestTime': 'October 14, 2021',
 'latestUpdate': 1694854742692,

In [ ]:
price = data['latestPrice']
market_cap = data["marketCap"]

In [ ]:
my_cols = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy (Equal Weighted)', 'Number of Shares to Buy (Market Cap Weighted)']
final_df = pd.DataFrame(columns=my_cols)

In [ ]:
final_df.append(
    pd.Series(
        [
         symbol,
         price,
         market_cap,
         'N/A',
         'N/A'
        ],
        index = my_cols
    ),
    ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy (Equal Weighted),Number of Shares to Buy (Market Cap Weighted)
0,AAPL,148.93,2468531289639,N/A,N/A


In [ ]:
final_df = pd.DataFrame(columns=my_cols)

In [ ]:
for tck in stocks['Ticker']:
  data = requests.get(f"https://sandbox.iexapis.com/stable/stock/{tck}/quote/?token={IEX_CLOUD_API_TOKEN}").json()
  final_df = final_df.append(
    pd.Series(
        [
         tck,
         data['latestPrice'],
         data["marketCap"],
         'N/A',
         'N/A'
        ],
        index = my_cols
    ),
    ignore_index = True
)

In [ ]:
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy (Equal Weighted),Number of Shares to Buy (Market Cap Weighted)
0,A,156.93,46477626907,N/A,N/A
1,AAL,19.81,12728021039,N/A,N/A
2,AAP,223.36,13961064013,N/A,N/A
3,AAPL,149.85,2412723097450,N/A,N/A
4,ABBV,113.23,195284131999,N/A,N/A
...,...,...,...,...,...
500,YUM,126.01,38314801364,N/A,N/A
501,ZBH,149.94,31528765260,N/A,N/A
502,ZBRA,518.48,27417891991,N/A,N/A
503,ZION,64.12,10237689731,N/A,N/A


In [ ]:
#batch api calls
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'], 100))

In [ ]:
symbol_strings = []
for i in range(len(symbol_groups)):
  symbol_strings.append(','.join(symbol_groups[i]))

In [ ]:
final_df = pd.DataFrame(columns=my_cols)

for symbol_str in symbol_strings:
  batch_api_url = f"https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_str}&token={IEX_CLOUD_API_TOKEN}"
  # print(batch_api_url)
  data = requests.get(batch_api_url).json()
  for symbol in symbol_str.split(','):
    final_df = final_df.append(
        pd.Series(
            [
             symbol,
             data[symbol]['quote']['latestPrice'],
             data[symbol]['quote']['marketCap'],
             'N/A',
             'N/A'
            ],
            index = my_cols
        ),
        ignore_index =True
    )

final_df = final_df.fillna(value = 0)

In [ ]:
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy (Equal Weighted),Number of Shares to Buy (Market Cap Weighted)
0,A,159.72,46643106943,N/A,N/A
1,AAL,19.98,12734951886,N/A,N/A
2,AAP,222.63,13859642336,N/A,N/A
3,AAPL,147.38,2427729651898,N/A,N/A
4,ABBV,112.76,201049322393,N/A,N/A
...,...,...,...,...,...
500,YUM,125.53,38042723290,N/A,N/A
501,ZBH,152.40,31362471016,N/A,N/A
502,ZBRA,529.15,28527870113,N/A,N/A
503,ZION,64.66,10387080764,N/A,N/A


In [ ]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:1000000


In [ ]:
position_size = float(portfolio_size) / len(final_df.index)
for i in range(0, len(final_df['Ticker'])):
    final_df.loc[i, 'Number of Shares to Buy (Equal Weighted)'] = math.floor(position_size / final_df['Stock Price'][i])
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy (Equal Weighted),Number of Shares to Buy (Market Cap Weighted)
0,A,159.72,46643106943,12,0
1,AAL,19.98,12734951886,99,0
2,AAP,222.63,13859642336,8,0
3,AAPL,147.38,2427729651898,13,3
4,ABBV,112.76,201049322393,17,0
...,...,...,...,...,...
500,YUM,125.53,38042723290,15,0
501,ZBH,152.40,31362471016,12,0
502,ZBRA,529.15,28527870113,3,0
503,ZION,64.66,10387080764,30,0


In [ ]:
total_cap = sum(final_df['Market Capitalization'])

In [ ]:
for i in range(0, len(final_df['Ticker'])):
  buy = ((final_df.iloc[i]['Market Capitalization']/total_cap)*float(portfolio_size)) / final_df.iloc[i]['Stock Price']
  final_df.loc[i, 'Number of Shares to Buy (Market Cap Weighted)'] = int(buy)

In [ ]:
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy (Equal Weighted),Number of Shares to Buy (Market Cap Weighted)
0,A,159.72,46643106943,12,6
1,AAL,19.98,12734951886,99,15
2,AAP,222.63,13859642336,8,1
3,AAPL,147.38,2427729651898,13,392
4,ABBV,112.76,201049322393,17,42
...,...,...,...,...,...
500,YUM,125.53,38042723290,15,7
501,ZBH,152.40,31362471016,12,4
502,ZBRA,529.15,28527870113,3,1
503,ZION,64.66,10387080764,30,3


In [ ]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_df.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [ ]:

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy (Equal Weighted)', integer_format],
                    'E': ['Number of Shares to Buy (Market Cap Weighted)', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [ ]:
writer.save()

##Momentum Based Trading

In [ ]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

In [ ]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'

In [ ]:
data = requests.get(api_url).json()

In [ ]:
data

{'avg10Volume': 73105597,
 'avg30Volume': 84067591,
 'beta': 1.417209649677814,
 'companyName': 'Apple Inc',
 'day200MovingAvg': 141.15,
 'day30ChangePercent': -0.05932279066355939,
 'day50MovingAvg': 151.17,
 'day5ChangePercent': -0.00784490973164872,
 'dividendYield': 0.006105834423722812,
 'employees': 154075,
 'exDividendDate': '2021-07-30',
 'float': 0,
 'marketcap': 2384195823274,
 'maxChangePercent': 54.92020490065587,
 'month1ChangePercent': -0.058446910908852866,
 'month3ChangePercent': -0.03218676551262437,
 'month6ChangePercent': 0.0526546598753412,
 'nextDividendDate': '',
 'nextEarningsDate': '2021-10-19',
 'peRatio': 28.128998427000774,
 'sharesOutstanding': 17345438637,
 'ttmDividendRate': 0.8896310157328156,
 'ttmEPS': 5.2,
 'week52change': 0.17514074034334548,
 'week52high': 157.32,
 'week52highSplitAdjustOnly': 158.1,
 'week52low': 106.8,
 'week52lowSplitAdjustOnly': 108.99,
 'year1ChangePercent': 0.17377770926618114,
 'year2ChangePercent': 1.4416614779545074,
 'year5

In [ ]:
data['year1ChangePercent']

0.17377770926618114

In [ ]:
#batch api calls
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strs = []

for i in range(len(symbol_groups)):
  symbol_strs.append(','.join(symbol_groups[i]))

In [ ]:
my_cols = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns= my_cols)

for symbol_str in symbol_strs:
  batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_str}&token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(batch_api_url).json()

  for symbol in symbol_str.split(','):
    final_df = final_df.append(
        pd.Series(
            [
             symbol,
             data[symbol]['quote']['latestPrice'],
             data[symbol]['stats']['year1ChangePercent'],
             'N/A'
            ],
            index = my_cols
        ),
        ignore_index = True
    )

In [ ]:
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,155.36,0.446162,N/A
1,AAL,20.03,0.60833,N/A
2,AAP,216.68,0.356132,N/A
3,AAPL,144.51,0.178519,N/A
4,ABBV,109.61,0.30857,N/A
...,...,...,...,...
500,YUM,125.52,0.31423,N/A
501,ZBH,151.22,-0.0082106,N/A
502,ZBRA,521.02,0.734717,N/A
503,ZION,62.71,1.12348,N/A


In [ ]:
final_df.sort_values('One-Year Price Return', ascending = False, inplace = True)

In [ ]:
top_50_momentum = final_df[:50]
top_50_momentum.reset_index(drop = True, inplace = True)

In [ ]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [ ]:
position_size = float(portfolio_size) / len(top_50_momentum.index)
for i in range(0, len(top_50_momentum['Ticker'])):
  top_50_momentum['Number of Shares to Buy'].loc[i] = int(position_size / top_50_momentum['Price'].loc[i])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
top_50_momentum

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,39.94,3.33726,5007
1,MRO,16.24,2.89981,12315
2,FANG,112.61,2.68751,1776
3,LB,83.70,2.35769,2389
4,OXY,33.55,2.11648,5961
5,MCHP,73.81,1.60973,2709
6,SIVB,686.10,1.52143,291
7,EOG,94.70,1.48974,2111
8,IT,320.96,1.44636,623
9,ALB,242.86,1.41195,823


##Momentum Based Trading Improved

In [ ]:
!pip install xlsxwriter

In [ ]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

In [ ]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

In [ ]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [ ]:
#batch api calls
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strs = []

for i in range(len(symbol_groups)):
  symbol_strs.append(','.join(symbol_groups[i]))

In [ ]:
for symbol_string in symbol_strs:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)

In [ ]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,160.60,N/A,0.468284,N/A,0.162517,N/A,0.0293154,N/A,-0.115505,N/A,N/A
1,AAL,20.64,N/A,0.591731,N/A,-0.127268,N/A,-0.048407,N/A,0.0224483,N/A,N/A
2,AAP,215.72,N/A,0.392326,N/A,0.155936,N/A,0.0277526,N/A,0.0757432,N/A,N/A
3,AAPL,149.67,N/A,0.194571,N/A,0.0936773,N/A,-0.0352978,N/A,-0.0296556,N/A,N/A
4,ABBV,113.13,N/A,0.338477,N/A,0.0545424,N/A,-0.0623483,N/A,0.0359101,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,130.54,N/A,0.323827,N/A,0.0672437,N/A,0.073788,N/A,-0.0311615,N/A,N/A
501,ZBH,151.00,N/A,0.00578574,N/A,-0.135362,N/A,-0.0632947,N/A,-0.0129125,N/A,N/A
502,ZBRA,532.72,N/A,0.814125,N/A,0.0222025,N/A,-0.0366253,N/A,-0.116616,N/A,N/A
503,ZION,62.83,N/A,1.1794,N/A,0.129598,N/A,0.232602,N/A,0.105452,N/A,N/A


In [ ]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

In [ ]:
hqm_dataframe.fillna(0, inplace = True)

In [ ]:
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

In [ ]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,160.60,N/A,0.468284,0.659406,0.162517,0.734653,0.029315,0.532673,-0.115505,0.0158416,N/A
1,AAL,20.64,N/A,0.591731,0.744554,-0.127268,0.10099,-0.048407,0.20396,0.022448,0.586139,N/A
2,AAP,215.72,N/A,0.392326,0.570297,0.155936,0.724752,0.027753,0.524752,0.075743,0.807921,N/A
3,AAPL,149.67,N/A,0.194571,0.318812,0.093677,0.588119,-0.035298,0.253465,-0.029656,0.279208,N/A
4,ABBV,113.13,N/A,0.338477,0.518812,0.054542,0.493069,-0.062348,0.166337,0.035910,0.653465,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,130.54,N/A,0.323827,0.493069,0.067244,0.524752,0.073788,0.712871,-0.031161,0.273267,N/A
501,ZBH,151.00,N/A,0.005786,0.134653,-0.135362,0.0930693,-0.063295,0.164356,-0.012913,0.382178,N/A
502,ZBRA,532.72,N/A,0.814125,0.863366,0.022202,0.40396,-0.036625,0.247525,-0.116616,0.0138614,N/A
503,ZION,62.83,N/A,1.179400,0.952475,0.129598,0.655446,0.232602,0.956436,0.105452,0.887129,N/A


In [ ]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [ ]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index(drop = True, inplace = True)

In [ ]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MCHP,72.80,N/A,1.703047,0.990099,0.896989,1,1.050035,1,0.824001,1,0.997525
1,DVN,41.30,N/A,3.425679,1,0.820430,0.99604,0.485193,0.99802,0.420075,0.99604,0.997525
2,MRO,16.30,N/A,2.900311,0.99802,0.470582,0.988119,0.297171,0.986139,0.384967,0.994059,0.991584
3,FANG,113.56,N/A,2.798158,0.99604,0.384863,0.974257,0.296839,0.984158,0.447399,0.99802,0.988119
4,COP,76.44,N/A,1.242833,0.962376,0.427352,0.980198,0.293294,0.982178,0.301912,0.982178,0.976733
5,PXD,197.01,N/A,1.187546,0.954455,0.281221,0.922772,0.280917,0.980198,0.262941,0.980198,0.959406
6,SIVB,720.91,N/A,1.629535,0.988119,0.352855,0.964356,0.183423,0.928713,0.171243,0.952475,0.958416
7,OKE,66.87,N/A,1.391772,0.978218,0.305874,0.944554,0.204786,0.938614,0.225649,0.968317,0.957426
8,CF,61.00,N/A,1.136010,0.938614,0.298501,0.938614,0.228382,0.952475,0.324934,0.990099,0.95495
9,MOS,42.13,N/A,1.196981,0.956436,0.251438,0.873267,0.350585,0.990099,0.304422,0.984158,0.95099


In [ ]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")


In [ ]:
portfolio_input()

Enter the value of your portfolio:10000000


In [ ]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = int(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MCHP,72.80,2693,1.703047,0.990099,0.896989,1,1.050035,1,0.824001,1,0.997525
1,DVN,41.30,4747,3.425679,1,0.820430,0.99604,0.485193,0.99802,0.420075,0.99604,0.997525
2,MRO,16.30,12029,2.900311,0.99802,0.470582,0.988119,0.297171,0.986139,0.384967,0.994059,0.991584
3,FANG,113.56,1726,2.798158,0.99604,0.384863,0.974257,0.296839,0.984158,0.447399,0.99802,0.988119
4,COP,76.44,2565,1.242833,0.962376,0.427352,0.980198,0.293294,0.982178,0.301912,0.982178,0.976733
5,PXD,197.01,995,1.187546,0.954455,0.281221,0.922772,0.280917,0.980198,0.262941,0.980198,0.959406
6,SIVB,720.91,271,1.629535,0.988119,0.352855,0.964356,0.183423,0.928713,0.171243,0.952475,0.958416
7,OKE,66.87,2932,1.391772,0.978218,0.305874,0.944554,0.204786,0.938614,0.225649,0.968317,0.957426
8,CF,61.00,3214,1.136010,0.938614,0.298501,0.938614,0.228382,0.952475,0.324934,0.990099,0.95495
9,MOS,42.13,4654,1.196981,0.956436,0.251438,0.873267,0.350585,0.990099,0.304422,0.984158,0.95099


In [ ]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0000',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', float_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [ ]:
writer.save()

##Performance to Price Ratio Trading

In [ ]:
!pip install xlsxwriter

     |████████████████████████████████| 148 kB 5.1 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

In [ ]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [ ]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'

In [ ]:
data = requests.get(api_url).json()
data

{'avgTotalVolume': 81921238,
 'calculationPrice': 'close',
 'change': 1.1,
 'changePercent': 0.00786,
 'close': 146.29,
 'closeSource': 'fflaoiic',
 'closeTime': 1702896595421,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 150.23,
 'delayedPriceTime': 1697354381763,
 'extendedChange': 0.14,
 'extendedChangePercent': 0.0009,
 'extendedPrice': 146.56,
 'extendedPriceTime': 1712467147723,
 'high': 150.88,
 'highSource': 'ridmecd eety 5u linpe1a',
 'highTime': 1654238173122,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 151.9,
 'iexCloseTime': 1697100902448,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 151.32,
 'iexOpenTime': 1687469582196,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'isUSMarketOpen': False,
 'lastTradeTime': 1665123022159,
 'latestPrice': 145.9,
 'latestSource': 'Close',
 'latestTime': 'October 15, 2021',
 'latestUpdate': 1695720152079,
 'latestVolume'

In [ ]:
price = data["latestPrice"]
pe_ratio = data["peRatio"]

In [ ]:
#batch api calls
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks["Ticker"],100))
symbol_strings = []
for symbol_group in symbol_groups:
  symbol_strings.append(','.join(symbol_group))

In [ ]:
my_cols = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [ ]:
final_df = pd.DataFrame(columns=my_cols)

In [ ]:
for symbol_string in symbol_strings:
  batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(batch_api_url).json()
  for symbol in symbol_string.split(","):
    final_df = final_df.append(
        pd.Series(
            [
             symbol,
             data[symbol]["quote"]["latestPrice"],
             data[symbol]["quote"]["peRatio"],
             "N/A"
            ],
            index = my_cols
        ),
        ignore_index = True
    )

In [ ]:
final_df = final_df[final_df["Price-to-Earnings Ratio"] > 0]
final_df.sort_values("Price-to-Earnings Ratio", inplace = True)
final_df = final_df[:51]
final_df.reset_index(drop = True, inplace = True)

In [ ]:
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,78.60,4.12,N/A
1,BIO,746.93,5.4,N/A
2,MPC,67.50,5.86,N/A
3,PRU,115.89,6.24,N/A
4,ZION,64.28,6.29,N/A
5,AFL,57.22,6.8,N/A
6,GM,60.00,6.8,N/A
7,COF,168.97,6.9,N/A
8,CINF,121.99,6.94,N/A
9,COO,403.84,6.98,N/A


In [ ]:
def portfolio_input():
  global portfolio_size
  portfolio_size = input("Enter the Value of your Portfolio: ")

  try:
    val = float(portfolio_size)
  except ValueError:
    print("That is not a number. \nPlease try again: ")
    portfolio_size = input("Enter the Value of your Portfolio: ")

In [ ]:
portfolio_input()

Enter the Value of your Portfolio: gdsf
That is not a number. 
Please try again: 
Enter the Value of your Portfolio: 10000000


In [ ]:
position_size = float(portfolio_size) / len(final_df)

In [ ]:
for i in range(len(final_df)):
  final_df["Number of Shares to Buy"].iloc[i] = int(position_size / final_df["Price"].iloc[i])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,78.60,4.12,2494
1,BIO,746.93,5.4,262
2,MPC,67.50,5.86,2904
3,PRU,115.89,6.24,1691
4,ZION,64.28,6.29,3050
5,AFL,57.22,6.8,3426
6,GM,60.00,6.8,3267
7,COF,168.97,6.9,1160
8,CINF,121.99,6.94,1607
9,COO,403.84,6.98,485


##Performance to Price Ratio Trading (Improved)

In [1]:
!pip install xlsxwriter

     |████████████████████████████████| 148 kB 4.9 MB/s 


In [26]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy.stats import percentileofscore as score

In [3]:
stocks = pd.read_csv("sp_500_stocks.csv")

In [5]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [6]:
stocks_list = list(chunks(stocks["Ticker"],100))

In [8]:
stocks_strings = []
for stock in stocks_list:
  stocks_strings.append(",".join(stock))

In [10]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_df = pd.DataFrame(columns = rv_columns)

In [12]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [18]:
for stocks_string in stocks_strings:
  batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={stocks_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(batch_api_call_url).json()
  for symbol in stocks_string.split(","):
    enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
    ebitda = data[symbol]['advanced-stats']['EBITDA']
    gross_profit = data[symbol]['advanced-stats']['grossProfit']
    
    try:
        ev_to_ebitda = enterprise_value/ebitda
    except TypeError:
        ev_to_ebitda = np.NaN
    
    try:
        ev_to_gross_profit = enterprise_value/gross_profit
    except TypeError:
        ev_to_gross_profit = np.NaN

    rv_df = rv_df.append(
        pd.Series(
            [
             symbol,
             data[symbol]['quote']['latestPrice'],
             'N/A',
             data[symbol]['quote']['peRatio'],
             'N/A',
             data[symbol]['advanced-stats']['priceToBook'],
             'N/A',
             data[symbol]['advanced-stats']['priceToSales'],
             'N/A',
             ev_to_ebitda,
             'N/A',
             ev_to_gross_profit,
             'N/A',
             'N/A'
            ],
            index = rv_columns
        ),
        ignore_index = True
    )

In [28]:
rv_df.isna().sum()

Ticker                      0
Price                       0
Number of Shares to Buy     0
Price-to-Earnings Ratio     8
PE Percentile               0
Price-to-Book Ratio        15
PB Percentile               0
Price-to-Sales Ratio       15
PS Percentile               0
EV/EBITDA                  15
EV/EBITDA Percentile        0
EV/GP                      15
EV/GP Percentile            0
RV Score                    0
dtype: int64

In [31]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_df[column].fillna(rv_df[column].mean(), inplace = True)

In [32]:
rv_df.isna().sum()

Ticker                     0
Price                      0
Number of Shares to Buy    0
Price-to-Earnings Ratio    0
PE Percentile              0
Price-to-Book Ratio        0
PB Percentile              0
Price-to-Sales Ratio       0
PS Percentile              0
EV/EBITDA                  0
EV/EBITDA Percentile       0
EV/GP                      0
EV/GP Percentile           0
RV Score                   0
dtype: int64

In [33]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

In [36]:
for row in rv_df.index:
    for metric in metrics.keys():
        rv_df.loc[row, metrics[metric]] = score(rv_df[metric], rv_df.loc[row, metric])/100

#Print the entire DataFrame    
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,156.49,N/A,49.26,0.835644,9.46,0.792079,7.920,0.819802,30.995387,0.881188,14.508956,0.825743,N/A
1,AAL,20.62,N/A,-2.06,0.0752475,-1.69,0.049505,0.723,0.0673267,-4.728346,0.0257426,1.894434,0.0792079,N/A
2,AAP,220.69,N/A,23.31,0.439604,4.28,0.544554,1.300,0.164356,11.621070,0.328713,2.844867,0.130693,N/A
3,AAPL,150.66,N/A,28.90,0.574257,37.47,0.960396,6.900,0.784158,22.294359,0.718812,17.489145,0.881188,N/A
4,ABBV,111.63,N/A,30.46,0.59802,15.76,0.893069,3.700,0.519802,10.800861,0.291089,7.850018,0.508911,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.58,N/A,29.68,0.590099,-4.69,0.0455446,6.100,0.745545,21.952708,0.710891,10.210495,0.675248,N/A
501,ZBH,147.23,N/A,33.47,0.649505,2.55,0.342574,3.880,0.554455,20.303627,0.679208,6.654195,0.413861,N/A
502,ZBRA,517.18,N/A,38.25,0.733663,10.83,0.821782,5.530,0.679208,27.202168,0.825743,11.801596,0.740594,N/A
503,ZION,62.28,N/A,6.34,0.0891089,1.38,0.131683,2.910,0.425743,5.026621,0.0534653,2.737436,0.126733,N/A


In [38]:
from statistics import mean

for row in rv_df.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_df.loc[row, metrics[metric]])
    rv_df.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,156.49,N/A,49.26,0.835644,9.46,0.792079,7.920,0.819802,30.995387,0.881188,14.508956,0.825743,0.830891
1,AAL,20.62,N/A,-2.06,0.0752475,-1.69,0.049505,0.723,0.0673267,-4.728346,0.0257426,1.894434,0.0792079,0.0594059
2,AAP,220.69,N/A,23.31,0.439604,4.28,0.544554,1.300,0.164356,11.621070,0.328713,2.844867,0.130693,0.321584
3,AAPL,150.66,N/A,28.90,0.574257,37.47,0.960396,6.900,0.784158,22.294359,0.718812,17.489145,0.881188,0.783762
4,ABBV,111.63,N/A,30.46,0.59802,15.76,0.893069,3.700,0.519802,10.800861,0.291089,7.850018,0.508911,0.562178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.58,N/A,29.68,0.590099,-4.69,0.0455446,6.100,0.745545,21.952708,0.710891,10.210495,0.675248,0.553465
501,ZBH,147.23,N/A,33.47,0.649505,2.55,0.342574,3.880,0.554455,20.303627,0.679208,6.654195,0.413861,0.527921
502,ZBRA,517.18,N/A,38.25,0.733663,10.83,0.821782,5.530,0.679208,27.202168,0.825743,11.801596,0.740594,0.760198
503,ZION,62.28,N/A,6.34,0.0891089,1.38,0.131683,2.910,0.425743,5.026621,0.0534653,2.737436,0.126733,0.165347


In [40]:
rv_df.sort_values(by = 'RV Score', inplace = True)
rv_df = rv_df[:50]
rv_df.reset_index(drop = True, inplace = True)

In [41]:
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.10,N/A,8.17,0.118812,0.5364,0.0514851,0.4220,0.0227723,3.553124,0.0376238,0.418959,0.00990099,0.0481188
1,PRU,115.64,N/A,6.23,0.0871287,0.6987,0.0574257,0.6968,0.0574257,5.024408,0.0514851,0.670342,0.0138614,0.0534653
2,AAL,20.62,N/A,-2.06,0.0752475,-1.6900,0.049505,0.7230,0.0673267,-4.728346,0.0257426,1.894434,0.0792079,0.0594059
3,KSS,46.57,N/A,10.25,0.153465,1.3500,0.121782,0.3939,0.0178218,4.084137,0.039604,1.146733,0.0356436,0.0736634
4,MCK,205.69,N/A,-7.37,0.0594059,-60.3000,0.0138614,0.1349,0.00792079,8.323292,0.182178,2.891223,0.136634,0.08
5,L,58.39,N/A,10.46,0.164356,0.8316,0.0693069,1.0000,0.109901,5.485267,0.0613861,0.988241,0.029703,0.0869307
6,BA,217.75,N/A,-14.33,0.0514851,-7.7200,0.0435644,2.1500,0.330693,-29.795753,0.00792079,-206.453520,0.0039604,0.0875248
7,ALL,131.12,N/A,10.40,0.158416,1.4900,0.152475,0.8090,0.0772277,2.393273,0.0336634,0.772867,0.0178218,0.0879208
8,LNC,73.40,N/A,10.33,0.156436,0.6289,0.0534653,0.7040,0.0594059,8.010632,0.166337,0.682026,0.0158416,0.090297
9,HPQ,28.39,N/A,9.04,0.130693,-8.3000,0.039604,0.5295,0.0415842,6.832303,0.112871,2.852188,0.132673,0.0914851


In [42]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [44]:
portfolio_input()

Enter the value of your portfolio:100000000


In [45]:
position_size = float(portfolio_size) / len(rv_df.index)
for i in range(0, len(rv_df['Ticker'])-1):
    rv_df.loc[i, 'Number of Shares to Buy'] = int(position_size / rv_df['Price'][i])
rv_df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.10,71174,8.17,0.118812,0.5364,0.0514851,0.4220,0.0227723,3.553124,0.0376238,0.418959,0.00990099,0.0481188
1,PRU,115.64,17295,6.23,0.0871287,0.6987,0.0574257,0.6968,0.0574257,5.024408,0.0514851,0.670342,0.0138614,0.0534653
2,AAL,20.62,96993,-2.06,0.0752475,-1.6900,0.049505,0.7230,0.0673267,-4.728346,0.0257426,1.894434,0.0792079,0.0594059
3,KSS,46.57,42946,10.25,0.153465,1.3500,0.121782,0.3939,0.0178218,4.084137,0.039604,1.146733,0.0356436,0.0736634
4,MCK,205.69,9723,-7.37,0.0594059,-60.3000,0.0138614,0.1349,0.00792079,8.323292,0.182178,2.891223,0.136634,0.08
5,L,58.39,34252,10.46,0.164356,0.8316,0.0693069,1.0000,0.109901,5.485267,0.0613861,0.988241,0.029703,0.0869307
6,BA,217.75,9184,-14.33,0.0514851,-7.7200,0.0435644,2.1500,0.330693,-29.795753,0.00792079,-206.453520,0.0039604,0.0875248
7,ALL,131.12,15253,10.40,0.158416,1.4900,0.152475,0.8090,0.0772277,2.393273,0.0336634,0.772867,0.0178218,0.0879208
8,LNC,73.40,27247,10.33,0.156436,0.6289,0.0534653,0.7040,0.0594059,8.010632,0.166337,0.682026,0.0158416,0.090297
9,HPQ,28.39,70447,9.04,0.130693,-8.3000,0.039604,0.5295,0.0415842,6.832303,0.112871,2.852188,0.132673,0.0914851


In [52]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_df.to_excel(writer, sheet_name='Value Strategy', index = False)

In [53]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0000',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [54]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', float_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [55]:
writer.save()